### Objetivo do Projeto

Este notebook realiza o processo completo de ETL (Extração, Transformação e Carga) para limpar, padronizar e unificar as bases de dados de clientes da **Empresa Confidencial**.

**Objetivo:** Atribuir um ID único da `Base_Referencia` a todos os clientes das demais bases, utilizando *fuzzy matching* para lidar com inconsistências de nomes.



In [ ]:
# Aqui, importamos todas as bibliotecas que serão usadas no projeto.
import pandas as pd
import re
from fuzzywuzzy import process
import os


ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

## Funções de Apoio

Nesta seção, definimos as funções que nos ajudarão a limpar os nomes dos clientes e a encontrar as correspondências. Manter a lógica em funções torna o código mais limpo e reutilizável.

In [ ]:
# Célula das Funções

def limpar_nome_cliente(nome_original):
    """
    Aplica as regras de limpeza e padronização nos nomes dos clientes,
    seguindo as diretrizes definidas no projeto.
    """
    if not isinstance(nome_original, str):
        return ""

    nome_processado = nome_original.upper()

    # Regra Específica: Trata o caso "MDS – FASTSHOP"
    if 'MDS' in nome_processado and 'FASTSHOP' in nome_processado:
        return 'FASTSHOP'

    # Regra Geral: Para nomes com hífen (Ex: "CARGILL - BARREIRAS"),
    # pega apenas a parte antes do hífen.
    if '-' in nome_processado:
        nome_processado = nome_processado.split('-')[0].strip()

    # Regra de Limpeza: Remove apenas as aspas (duplas e simples)
    nome_processado = nome_processado.replace('"', '').replace("'", "")
    
    # Remove espaços em branco extras
    nome_processado = " ".join(nome_processado.split())
    
    return nome_processado

def encontrar_correspondencia(nome_cliente, lista_nomes_referencia, limite_pontuacao=85):
    """
    Compara um nome de cliente com a lista de referência e encontra o mais parecido,
    respeitando um limite mínimo de pontuação.
    """
    if not nome_cliente or not lista_nomes_referencia:
        return None, 0
        
    melhor_correspondecia, pontuacao = process.extractOne(nome_cliente, lista_nomes_referencia)
    
    if pontuacao >= limite_pontuacao:
        return melhor_correspondecia, pontuacao
    else:
        return None, pontuacao

print("Funções 'limpar_nome_cliente' e 'encontrar_correspondencia' definidas.")

## Passo 1: Definição de Caminhos e Estrutura de Pastas

Definimos onde os dados brutos estão e onde o resultado final será salvo. O script também cria a pasta de destino caso ela não exista.

In [ ]:
# Célula de Configuração de Caminhos

caminho_dados_brutos = os.path.join('Data', 'Raw')
caminho_dados_processados = os.path.join('Data', 'Processed')
nome_arquivo_excel = 'Base de Dados Clientes.xlsx'
caminho_completo_excel = os.path.join(caminho_dados_brutos, nome_arquivo_excel)

# Cria a pasta de saída "Data/Processed" se ela não existir
os.makedirs(caminho_dados_processados, exist_ok=True)
caminho_arquivo_saida = os.path.join(caminho_dados_processados, "base_unificada_clientes_beecorp.csv")

print(f"Caminho do arquivo de entrada: {caminho_completo_excel}")
print(f"Caminho do arquivo de saída: {caminho_arquivo_saida}")

## Passo 2: Extração (E) - Carregamento dos Dados

Lemos os dados a partir das abas do arquivo Excel para a memória, utilizando a biblioteca Pandas.

In [ ]:
# Célula de Carregamento dos Dados

try:
    nome_aba_referencia = 'Base_Referencia'
    nomes_abas_clientes = [
        'Recrutamento_Selecao', 'Profissional_Ponta',
        'Operacoes', 'Departamento_Pessoal'
    ]

    base_referencia = pd.read_excel(caminho_completo_excel, sheet_name=nome_aba_referencia)

    lista_dfs_clientes = [
        pd.read_excel(caminho_completo_excel, sheet_name=nome_aba) for nome_aba in nomes_abas_clientes
    ]
    print("Todas as abas do Excel foram carregadas com sucesso!")

except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo: {e}")

# Visualiza as primeiras 5 linhas da base de referência para verificação
base_referencia.head()


Processando a base: Departamento_Pessoal
   -> Padronização de clientes concluída.

Processando a base: Operacoes
   -> Padronização de clientes concluída.

Processando a base: Profissional_Ponta
   -> Padronização de clientes concluída.

Processando a base: Recrutamento_Selecao
   -> Padronização de clientes concluída.

--- Processo de Unificação Concluído ---
Total de registros na base unificada: 2281
Total de clientes únicos padronizados: 10
Base de dados unificada salva em: ../Data/Processed/base_padronizada_final.xlsx


## Passo 3: Transformação (T) - Limpeza e Unificação

Esta é a etapa principal do processo. Aqui, nós:
1.  Juntamos todos os nomes de clientes em uma lista única.
2.  Preparamos a base de referência.
3.  Aplicamos a limpeza e o *fuzzy matching* para encontrar as correspondências.
4.  Fazemos o `merge` (join) para unificar os dados e atribuir os IDs.

In [ ]:
# Célula de Transformação e Processamento

# 1. Juntar todos os clientes em uma lista única
todos_os_clientes = pd.concat([df["Nome do Cliente"] for df in lista_dfs_clientes])
clientes_unicos = pd.DataFrame(todos_os_clientes.unique(), columns=['Nome Original']).dropna()
print(f"Encontrados {len(clientes_unicos)} nomes de clientes únicos.")

# 2. Preparar a base de referência
base_referencia['Razão Social Limpa'] = base_referencia['Razão Social'].apply(limpar_nome_cliente)
lista_referencia = base_referencia['Razão Social Limpa'].unique().tolist()
print("Base de referência preparada.")

# 3. Limpar nomes e encontrar correspondências
clientes_unicos['Nome Limpo'] = clientes_unicos['Nome Original'].apply(limpar_nome_cliente)
resultados_match = clientes_unicos['Nome Limpo'].apply(
    lambda nome: pd.Series(encontrar_correspondencia(nome, lista_referencia), index=['Match Encontrado', 'Pontuacao'])
)
clientes_unicos_com_match = pd.concat([clientes_unicos, resultados_match], axis=1)
print("Processo de fuzzy matching concluído.")

# 4. Unificar com a base de referência
base_unificada = pd.merge(
    clientes_unicos_com_match, base_referencia,
    left_on='Match Encontrado', right_on='Razão Social Limpa',
    how='left'
)
print("Merge finalizado.")

# Visualiza as primeiras linhas do resultado com as pontuações
base_unificada.head()

## Passo 4: Carga (L) - Organização e Salvamento

Na etapa final, selecionamos as colunas mais importantes, organizamos o resultado e salvamos em um novo arquivo CSV na pasta `Data/Processed`.

In [ ]:
# Célula de Salvamento

# 1. Organizar colunas
base_final_organizada = base_unificada[[
    'Nome Original', 'Nome Limpo', 'Razão Social', 'Pontuacao',
    'ID', 'Unidade', 'Empresa / Grupo Economico'
]]

# 2. Salvar em CSV
base_final_organizada.to_csv(caminho_arquivo_saida, index=False, encoding='utf-8-sig')

# 3. Análise final
total_clientes = len(base_final_organizada)
clientes_com_id = base_final_organizada['ID'].notna().sum()
percentual_sucesso = (clientes_com_id / total_clientes) * 100 if total_clientes > 0 else 0

print("--- Análise de Resultados ---")
print(f"Processo concluído com sucesso!")
print(f"Total de clientes únicos processados: {total_clientes}")
print(f"Clientes que receberam um ID: {clientes_com_id}")
print(f"Taxa de sucesso na unificação: {percentual_sucesso:.2f}%")
print(f"\nArquivo final salvo em: {caminho_arquivo_saida}")